In [1]:
# Google Colab specific setup for Kaggle API
!pip install kaggle
!mkdir /content/.kaggle
!touch /content/.kaggle/kaggle.json
!echo {\"username\":\"kevinkoh\",\"key\":\"9e5db5af368dd7342bc837352adc8bb4\"} > /content/.kaggle/kaggle.json
!chmod 600 /content/.kaggle/kaggle.json
!kaggle competitions download -c titanic -p '/input' 
!mv /input/titanic/* /input/

  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /content/.cache/pip/wheels/56/4a/b5/93df30500029d78108fdfaace4f08fb881d27903f795a6367e
Successfully built kaggle
train.csv: Downloaded 60KB of 60KB
test.csv: Downloaded 28KB of 28KB
gender_submission.csv: Downloaded 3KB of 3KB


# 1. Data Wraggling
## 1.1 Import Libraries

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

#load packages
import sys #access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features
print("pandas version: {}". format(pd.__version__))

import matplotlib #collection of functions for scientific and publication-ready visualization
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import scipy as sp #collection of functions for scientific computing and advance mathematics
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
print("IPython version: {}". format(IPython.__version__)) 

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Python version: 3.6.3 (default, Oct  3 2017, 21:45:48) 
[GCC 7.2.0]
pandas version: 0.22.0
matplotlib version: 2.1.2
NumPy version: 1.14.0
SciPy version: 0.19.1
IPython version: 5.5.0
scikit-learn version: 0.19.1
-------------------------
gender_submission.csv
test.csv
titanic
train.csv



## 1.2 Data Import  
Since we have a total of 891 rows of data, we should be expecting 891 count.  We can see that Age, Cabin and Embarked seems to be missing for some rows.   

Futher more using the info(), we can see which of the colums are numeric and non-numeric from their data types.

In [3]:
# Import train and test data
train_df = pd.read_csv('../input/train.csv')
pred_df  = pd.read_csv('../input/test.csv')

# Passing by reference to clean both datasets at the same time
data_cleaner = [train_df, pred_df]

# Preview data
print (train_df.info())
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Simillarly, we want to generate summary figures for the non-numeric columns.

When dealing with non-numeric columns, we generally try to turn them into factors/categories with few levels.

* We can see that Name are entirely unique, it may not be immediately useful as it has too many unique values. We may explore extracting First and Last names in the later analysis.
* Sex has two values as it should be (Male and Female), with more males on board than females.
* Ticket may not be entirely unique, with some duplicates. However, with 681 unque values, it may become difficult to categorise them.
* Cabin has more than have the total rows without a value.
* Embared has a nice 3 unique values and just 2 missing value. We may want to fill in the missing value.

In [4]:
train_df.describe(include='all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Moran, Miss. Bertha",male,NaN,NaN,NaN,1601,NaN,C23 C25 C27,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [5]:
pred_df.describe(include='all')

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,418.000000,418.000000,418,418,332.000000,418.000000,418.000000,418,417.000000,91,418
unique,NaN,NaN,418,2,NaN,NaN,NaN,363,NaN,76,3
top,NaN,NaN,"Johansson Palmquist, Mr. Oskar Leander",male,NaN,NaN,NaN,PC 17608,NaN,B57 B59 B63 B66,S
freq,NaN,NaN,1,266,NaN,NaN,NaN,5,NaN,3,270
mean,1100.500000,2.265550,NaN,NaN,30.272590,0.447368,0.392344,NaN,35.627188,NaN,NaN
std,120.810458,0.841838,NaN,NaN,14.181209,0.896760,0.981429,NaN,55.907576,NaN,NaN
min,892.000000,1.000000,NaN,NaN,0.170000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,996.250000,1.000000,NaN,NaN,21.000000,0.000000,0.000000,NaN,7.895800,NaN,NaN
50%,1100.500000,3.000000,NaN,NaN,27.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,1204.750000,3.000000,NaN,NaN,39.000000,1.000000,0.000000,NaN,31.500000,NaN,NaN


# 2. Data Pipeline
Setup a proper pipeline so that all data (training and testing) undergo the same wraggling.

## 2.1 Missing Data
Lets impute missing data with the median.
We will fill in Age and Embarked as they have few rows of missing data.

However for Cabin more than half of it is missing, it may not be advisable to fill it in with the median. 
Imputing by regressing it on other columns could be tried out later.

In [0]:
# Define a playing dataset
my_df = train_df.copy()

In [7]:
my_df.loc[:,'Age'].fillna(value = my_df['Age'].median(axis=0), inplace = True)
my_df['Age'].count()

891

In [8]:
my_df.loc[:,'Embarked'].fillna(value = my_df['Embarked'].mode()[0], inplace = True) 
my_df['Embarked'].count()

891

## 2.2 Feature Engineering

Let's try creating useful features that might help predict the survival status of each passenger.

Using the number of sibings, spouses, parents and children, we can derive the number of family members on board. Perhaps passengers with more family members are less likely to survive as they would spend more time trying to find and gather their family members.

In [9]:
my_df['Family'] = my_df['SibSp'] + my_df['Parch']
my_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


It looks like we could extract the first letter of the cabin that might be useful

In [0]:
my_df['CabinLetter'] = my_df['Cabin'].str.extract('([a-zA-Z])')
# Too much missing data to impute for cabin
# my_df.loc[:,'CabinLetter'].fillna(value = my_df['CabinLetter'].mode()[0], inplace = True) 

In [11]:
# Quick peak at the possible relationship of survivors and CabinLetter
pd.crosstab(my_df['Survived'],my_df['CabinLetter'])

CabinLetter,A,B,C,D,E,F,G,T
Survived,,,,,,,,
0,8,12,24,8,8,5,2,1
1,7,35,35,25,24,8,2,0


## 2.3 Defining the actual pipeline

In [0]:
#Data Pipeline
def wragle(data):
    # Impute missing data with median
    data.loc[:,'Age'].fillna(value = data['Age'].median(axis=0), inplace = True)
    data.loc[:,'Embarked'].fillna(value = data['Embarked'].mode()[0], inplace = True)
    data.loc[:,'Fare'].fillna(value = data['Fare'].mean(axis=0), inplace = True)
    # Feature Engineering
    data['Family'] = data['SibSp'] + data['Parch']
    data['CabinLetter'] = data['Cabin'].str.extract('([a-zA-Z])')
    # KIV other imputation methods?
    # data.loc[:,'CabinLetter'].fillna(value = data['CabinLetter'].mode()[0], inplace = True)
    # Dummycoding categorical variables
    # pd.get_dummies(train_df,columns = ['Embarked', 'Sex'],drop_first = True)
    data = pd.merge(data, pd.get_dummies(data['Embarked']), left_index = True, right_index = True)
    data = pd.merge(data, pd.get_dummies(data['Sex']), left_index = True, right_index = True)
    #data = pd.merge(data, pd.get_dummies(data['CabinLetter'], prefix = 'CL'), left_index = True, right_index = True)
    # chose not to impute Cabin as too much missing data
#     data.loc[:,'CabinLetter'].fillna(value = data['CabinLetter'].mode()[0], inplace = True)
    return(data)

In [0]:
# def normalise(data):
#     for col in data.loc[:,data.dtypes!='object'].columns:
#         data[col] = (data[col] - data[col].mean()) / (data[col].std())

In [0]:
train_df = wragle(train_df)
pred_df = wragle(pred_df)

In [14]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family,CabinLetter,C,Q,S,female,male
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,NaN,0,0,1,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,C,1,0,0,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,NaN,0,0,1,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,C,0,0,1,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,NaN,0,0,1,0,1


In [0]:
# Need to change to normalise only continuous variables
# size_train = len(train_df)
# combined = pd.concat([train_df, pred_df])
# combined = normalise(combined)
# train_df = combined.iloc[:size_train, :]
# pred_df = combined.iloc[size_train:, :]

In [16]:
# train_df.head()

,Age,C,Cabin,CabinLetter,Embarked,Family,Fare,Name,Parch,PassengerId,Pclass,Q,S,Sex,SibSp,Survived,Ticket,female,male
0,-0.575867,-0.509575,NaN,NaN,S,0.073324,-0.503437,"Braund, Mr. Owen Harris",-0.444829,-1.730067,0.841595,-0.321917,0.654761,male,0.481104,-0.788829,A/5 21171,-0.743213,0.743213
1,0.662976,1.960920,C85,C,C,0.073324,0.734187,"Cumings, Mrs. John Bradley (Florence Briggs Th...",-0.444829,-1.727422,-1.545507,-0.321917,-1.526109,female,0.481104,1.266279,PC 17599,1.344482,-1.344482
2,-0.266157,-0.509575,NaN,NaN,S,-0.558133,-0.490390,"Heikkinen, Miss. Laina",-0.444829,-1.724776,0.841595,-0.321917,0.654761,female,-0.478904,1.266279,STON/O2. 3101282,1.344482,-1.344482
3,0.430693,-0.509575,C123,C,S,0.073324,0.382744,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",-0.444829,-1.722131,-1.545507,-0.321917,0.654761,female,0.481104,1.266279,113803,1.344482,-1.344482
4,0.430693,-0.509575,NaN,NaN,S,-0.558133,-0.487974,"Allen, Mr. William Henry",-0.444829,-1.719485,0.841595,-0.321917,0.654761,male,-0.478904,-0.788829,373450,-0.743213,0.743213


# First model
Let's just run the first logistic regression model on the numeric variables just to how things are going (we are running out of time too). Categorical variables require some encoding before they can be modeled.

\begin{equation}
\sigma(t) = \frac{1}{1 + e^{(-t)}} \\
t = \beta_0 + \beta_1x_1 + ... + \beta_nx_n
\end{equation}

# Variable selection

In [0]:
# Select numeric variables
X = train_df[['Pclass', 'Age', 'Fare', 'female', 'male', 'Family'
              #,'C', 'Q', 'S', 
              #, 'CL_A', 'CL_B', 'CL_C', 'CL_D', 'CL_E', 'CL_F', 'CL_G', 'CL_T'
             ]]
Y = train_df['Survived']

In [0]:
# Linear model
from sklearn import linear_model
lm_model = linear_model.LogisticRegression(C=1e5)

In [0]:
# Decision Tree
from sklearn import tree
tree_model = tree.DecisionTreeClassifier(max_depth = 5, min_samples_split = 30)

## KFold Cross-Validation
This is done to obtain a good proxy to the accuracy score, with the KFold technique instead of the usual training and testing data splitting.

In [18]:
from sklearn.model_selection import KFold
v = KFold(n_splits=10)

fold_accuracy = []

for train, test in v.split(X):
    X_train = X.iloc[train]
    X_test = X.iloc[test]
    
    y_train = Y.iloc[train]
    y_test = Y.iloc[test]

    model = tree_model.fit(X_train, y_train)
    valid = model.score(X_test, y_test)
    
    fold_accuracy.append(valid)
    
print("Accuracy per fold: ", fold_accuracy, "\n")
print("Min accuracy: ", min(fold_accuracy))
print("Average accuracy: ", sum(fold_accuracy)/len(fold_accuracy))
print("Max accuracy: ", max(fold_accuracy))

#lm.fit(X_train, X_test)

Accuracy per fold:  [0.7555555555555555, 0.8314606741573034, 0.7752808988764045, 0.8426966292134831, 0.8089887640449438, 0.797752808988764, 0.7752808988764045, 0.7752808988764045, 0.8876404494382022, 0.8202247191011236] 

Min accuracy:  0.7555555555555555
Average accuracy:  0.807016229712859
Max accuracy:  0.8876404494382022


In [0]:
# Predict Survival of passenagers using Pclass, Age, Family and fare
X_pred = pred_df[['Pclass', 'Age', 'Fare', 'female', 'male','Family'
                  #,'C', 'Q', 'S',
                  #, 'CL_A', 'CL_B', 'CL_C', 'CL_D', 'CL_E', 'CL_F', 'CL_G'
                 ]]
Y_pred = model.predict(X_pred)
submission = pd.concat([pred_df[['PassengerId']],pd.DataFrame(Y_pred)],axis=1)
submission.columns = ['PassengerId', 'Survived']
submission.to_csv('submission.csv',index=False)

In [20]:
submission.groupby('Survived').count()

,PassengerId
Survived,
0,261
1,157


In [21]:
# Submit solutions
# !kaggle competitions submit -c titanic -f 'submission.csv'

usage: kaggle competitions submit [-h] -c COMPETITION -f FILE -m MESSAGE
kaggle competitions submit: error: the following arguments are required: -m/--message
